In [7]:
import cv2
import numpy as np
import time
from ultralytics import YOLO

# Load trained YOLOv8 model
model = YOLO(r"C:\Users\dhruv\Documents\RM Sem 2\Codes\dataset\prem\best.pt").to('cuda')

# Define class names
class_names = ["Physics Glitch", "Visual Glitch", "Frame Drop", "Texture Glitch", "Screen Freeze"]

# Adaptive screen freeze variables
INITIAL_FREEZE_THRESH = 100000
FREEZE_FRAME_THRESH = INITIAL_FREEZE_THRESH
MIN_FREEZE_DURATION = 5
MAX_FREEZE_DURATION = 50
freeze_counter = 0
freeze_detected = False
frame_diffs = []
LOG_FILE_PATH = "detection_log.txt"

def detect_frame_drop(prev_frame, current_frame, threshold=30):
    """Detects frame drops based on pixel differences."""
    if prev_frame is None:
        return False
    diff = cv2.absdiff(prev_frame, current_frame)
    non_zero_count = np.count_nonzero(diff)
    return non_zero_count < threshold

def process_video(video_path, output_path, log_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    prev_frame = None
    frame_count = 0
    
    with open(log_path, "w") as log_file:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break
            
            current_time = time.strftime('%Y-%m-%d %H:%M:%S')
            glitches = []
            debug_info = []
            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            
            frame_count += 1
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Run YOLOv8 inference
            results = model.predict(frame, batch=8)
            
            detections = []
            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = box.conf[0].item()
                    cls = int(box.cls[0].item())
                    class_label = class_names[cls]
                    
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    text = f"{class_label} ({conf:.2f})"
                    cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    detections.append(f"Frame {frame_count}: {class_label} detected at ({x1},{y1},{x2},{y2}) with {conf:.2f} confidence.")
            
            # Adaptive Screen Freeze Detection
            global FREEZE_FRAME_THRESH, freeze_counter, freeze_detected
            if prev_frame is not None:
                frame_diff = cv2.absdiff(frame_gray, prev_frame)
                diff_sum = np.sum(frame_diff)
                frame_diffs.append(diff_sum)
                if len(frame_diffs) > 50:
                    frame_diffs.pop(0)
                if frame_diffs:
                    avg_diff = np.mean(frame_diffs)
                    FREEZE_FRAME_THRESH = max(INITIAL_FREEZE_THRESH, avg_diff * 0.2)
                debug_info.append(f"Frame Diff: {diff_sum}")
                debug_info.append(f"Adaptive Threshold: {int(FREEZE_FRAME_THRESH)}")
                if diff_sum < FREEZE_FRAME_THRESH:
                    freeze_counter += 1
                    if MIN_FREEZE_DURATION <= freeze_counter <= MAX_FREEZE_DURATION:
                        glitches.append(f"Screen Freeze ({freeze_counter} frames)")
                        freeze_detected = True
                else:
                    if freeze_detected:
                        log_file.write(f"{frame_number}, {current_time}, {freeze_counter}, {diff_sum}\n")
                    freeze_counter = 0
                    freeze_detected = False
            
            # Display freeze detection info
            status_text = [
                f"Frame: {frame_number}",
                f"Time: {current_time}",
            ] + glitches + debug_info
            y_offset = 30
            for text in status_text:
                cv2.putText(frame, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                y_offset += 25
            
            prev_frame = frame_gray.copy()
            
            # Write logs
            for detection in detections:
                log_file.write(detection + "\n")
            
            out.write(frame)
            cv2.imshow("Glitch Detection", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Processed video saved at: {output_path}")
    print(f"Detection log saved at: {log_path}")

# Example Usage
video_path = r"C:\Users\dhruv\Documents\RM Sem 2\Codes\dataset\yt video for annote\videoplayback2.mp4"
output_path = "output_video1.mp4"
log_path = "detection_log1.txt"
process_video(video_path, output_path, log_path)



0: 384x640 (no detections), 28.9ms
Speed: 5.0ms preprocess, 28.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 2.1ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.3ms
Speed: 2.0ms preprocess, 12.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.3ms
Speed: 2.0ms preprocess, 13.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.5ms
Speed: 2.0ms preprocess, 13.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.0ms
Speed: 2.0ms preprocess, 9.0ms inf